In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%load_ext cudf.pandas
import cuml
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv', index_col='id')

In [3]:
train.shape

(750000, 11)

In [4]:
test.shape

(250000, 10)

## Data Understanding

In [5]:
train.sample(10)

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
id,,,,,,,,,,,
418548,Mystery Matters,Episode 90,52.46,True Crime,27.20,Wednesday,Afternoon,60.2,0.0,Positive,38.72000
217351,Game Day,Episode 70,24.81,Sports,36.29,Tuesday,Afternoon,12.72,3.0,Negative,10.97180
393607,Fitness First,Episode 88,80.38,Health,88.33,Wednesday,Morning,26.52,2.0,Negative,53.13332
230797,Criminal Minds,Episode 30,31.91,True Crime,85.03,Tuesday,Afternoon,98.67,2.0,Negative,19.68336
589937,Fitness First,Episode 41,22.18,Health,21.20,Friday,Morning,12.2,0.0,Negative,5.60613
173502,Life Lessons,Episode 95,53.04,Lifestyle,32.30,Monday,Evening,78.6,1.0,Neutral,51.79939
23700,Business Briefs,Episode 20,96.11,Business,83.73,Tuesday,Morning,65.83,0.0,Positive,88.57411
729521,Mystery Matters,Episode 3,106.66,True Crime,98.29,Tuesday,Evening,23.97,2.0,Neutral,52.58826
45733,Style Guide,Episode 90,12.82,Lifestyle,43.80,Thursday,Evening,<NA>,1.0,Negative,4.47776


In [6]:
train.describe()

,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,662907.000000,750000.000000,603970.000000,749999.000000,750000.000000
mean,64.504738,59.859901,52.236449,1.348855,45.437406
std,32.969603,22.873098,28.451241,1.151130,27.138306
min,0.000000,1.300000,0.000000,0.000000,0.000000
25%,35.730000,39.410000,28.380000,0.000000,23.178350
50%,63.840000,60.050000,53.580000,1.000000,43.379460
75%,94.070000,79.530000,76.600000,2.000000,64.811580
max,325.240000,119.460000,119.910000,103.910000,119.970000


In [7]:
train.shape

(750000, 11)

In [8]:
test.shape

(250000, 10)

In [9]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0   Podcast_Name                 750000 non-null  object
 1   Episode_Title                750000 non-null  object
 2   Episode_Length_minutes       662907 non-null  float64
 3   Genre                        750000 non-null  object
 4   Host_Popularity_percentage   750000 non-null  float64
 5   Publication_Day              750000 non-null  object
 6   Publication_Time             750000 non-null  object
 7   Guest_Popularity_percentage  603970 non-null  float64
 8   Number_of_Ads                749999 non-null  float64
 9   Episode_Sentiment            750000 non-null  object
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), object(6)
memory usage: 88.8+ MB


In [10]:
train.isnull().sum()

Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
dtype: int64

In [11]:
test.isnull().sum()

Podcast_Name                       0
Episode_Title                      0
Episode_Length_minutes         28736
Genre                              0
Host_Popularity_percentage         0
Publication_Day                    0
Publication_Time                   0
Guest_Popularity_percentage    48832
Number_of_Ads                      0
Episode_Sentiment                  0
dtype: int64

## Data Preparation

Reducing the memory space of the dataset by converting the num_cols into float32 and cat_cols into category

In [12]:
num_cols = test.select_dtypes(include=['float64']).columns
for col in num_cols:
    train[col] = train[col].astype('float32')
    test[col] = test[col].astype('float32')

In [13]:
cat_cols = train.select_dtypes(include=['object']).columns
for col in cat_cols:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

In [14]:
from scipy.stats.mstats import winsorize

train['Number_of_Ads'] = winsorize(train['Number_of_Ads'], limits=(0.01, 0.01))
test['Number_of_Ads'] = winsorize(test['Number_of_Ads'], limits=(0.01, 0.01))

train['Listening_Time_minutes'] = winsorize(train['Listening_Time_minutes'], limits=(0.01, 0.01))

In [15]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0   Podcast_Name                 750000 non-null  category
 1   Episode_Title                750000 non-null  category
 2   Episode_Length_minutes       662907 non-null  float32
 3   Genre                        750000 non-null  category
 4   Host_Popularity_percentage   750000 non-null  float32
 5   Publication_Day              750000 non-null  category
 6   Publication_Time             750000 non-null  category
 7   Guest_Popularity_percentage  603970 non-null  float32
 8   Number_of_Ads                750000 non-null  float32
 9   Episode_Sentiment            750000 non-null  category
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: category(6), float32(4), float64(1)
memory usage: 27.4 MB


In [16]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer(strategy='median')
num_cols = test.select_dtypes(include=['float32']).columns
train[num_cols] = impute.fit_transform(train[num_cols])
test[num_cols] = impute.fit_transform(test[num_cols])

In [17]:
sentiments = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
train['Episode_Sentiment'] = train['Episode_Sentiment'].map(sentiments).astype('float32')
test['Episode_Sentiment'] = test['Episode_Sentiment'].map(sentiments).astype('float32')

## Feature engineering

## Count Encoding for high cardinality columns

In [18]:
from category_encoders import CountEncoder

ce = CountEncoder()
count_category = ['Podcast_Name', 'Genre', 'Episode_Title']
train[count_category] = ce.fit_transform(train[count_category])
test[count_category] = ce.transform(test[count_category])

In [19]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ['Publication_Day', 'Publication_Time']
for colname in cat_cols:
    le = LabelEncoder()
    train[colname] = le.fit_transform(train[colname])
    test[colname] = le.transform(test[colname])

In [20]:
from sklearn.preprocessing import KBinsDiscretizer

kmeans = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
train['Kmeans_bins'] = kmeans.fit_transform(train[['Episode_Length_minutes']])
test['Kmeans_bins'] = kmeans.transform(test[['Episode_Length_minutes']])

In [21]:
from sklearn.preprocessing import KBinsDiscretizer

ew = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
train['Host_bins'] = ew.fit_transform(train[['Episode_Length_minutes']])
test['Host_bins'] = ew.transform(test[['Episode_Length_minutes']])

In [22]:
for col in ['Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Episode_Length_minutes']:
    for k in range(0, 3):
        new_round_col = f"{col}_round{k}"
        train[new_round_col] = train[col].round(k)
        test[new_round_col] = test[col].round(k)

In [23]:
train['Guest_Popularity_percentage'] =  train['Guest_Popularity_percentage'].clip(upper=100)
train['Host_Popularity_percentage'] =  train['Host_Popularity_percentage'].clip(upper=100)
test['Guest_Popularity_percentage'] =  test['Guest_Popularity_percentage'].clip(upper=100)
test['Host_Popularity_percentage'] =  test['Host_Popularity_percentage'].clip(upper=100)


In [24]:
train['Average_popularity'] = (train['Host_Popularity_percentage'] + train['Guest_Popularity_percentage']) / 2
test['Average_popularity'] = (test['Host_Popularity_percentage'] + test['Guest_Popularity_percentage']) / 2
train['minutes_per_ads'] = train['Episode_Length_minutes'] / (train['Number_of_Ads'] + 1e-3) 
test['minutes_per_ads'] = test['Episode_Length_minutes'] / (test['Number_of_Ads'] + 1e-3)

In [25]:
train.columns

Index(['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes', 'Kmeans_bins', 'Host_bins',
       'Host_Popularity_percentage_round0',
       'Host_Popularity_percentage_round1',
       'Host_Popularity_percentage_round2',
       'Guest_Popularity_percentage_round0',
       'Guest_Popularity_percentage_round1',
       'Guest_Popularity_percentage_round2', 'Episode_Length_minutes_round0',
       'Episode_Length_minutes_round1', 'Episode_Length_minutes_round2',
       'Average_popularity', 'minutes_per_ads'],
      dtype='object')

In [26]:
features = ['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment', 'Kmeans_bins', 'Host_bins',
       'Host_Popularity_percentage_round0',
       'Host_Popularity_percentage_round1',
       'Host_Popularity_percentage_round2',
       'Guest_Popularity_percentage_round0',
       'Guest_Popularity_percentage_round1',
       'Guest_Popularity_percentage_round2', 'Episode_Length_minutes_round0',
       'Episode_Length_minutes_round1', 'Episode_Length_minutes_round2',
       'Average_popularity', 'minutes_per_ads']

In [27]:
cat_features = ['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre', 'Publication_Day', 'Publication_Time']

In [28]:
interact = []
for i, c1 in enumerate(cat_features):
    for c2 in (cat_features[i+1:]):
        n = f'{c1}_{c2}'
        train[n] = train[c1] * train[c2]
        test[n] = test[c1] * test[c2]
        interact.append(n)

print(f"There are {len(interact)} interaction features:")

There are 15 interaction features:


In [29]:
train.columns

Index(['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes', 'Kmeans_bins', 'Host_bins',
       'Host_Popularity_percentage_round0',
       'Host_Popularity_percentage_round1',
       'Host_Popularity_percentage_round2',
       'Guest_Popularity_percentage_round0',
       'Guest_Popularity_percentage_round1',
       'Guest_Popularity_percentage_round2', 'Episode_Length_minutes_round0',
       'Episode_Length_minutes_round1', 'Episode_Length_minutes_round2',
       'Average_popularity', 'minutes_per_ads', 'Podcast_Name_Episode_Title',
       'Podcast_Name_Episode_Length_minutes', 'Podcast_Name_Genre',
       'Podcast_Name_Publication_Day', 'Podcast_Name_Publication_Time',
       'Episode_Title_Episode_Length_minutes', 'Episode_Title_Genre',
       'Episode_Title_Publication_Day', 'Episode_Ti

In [30]:
from sklearn.model_selection import KFold, cross_val_predict
from cuml.metrics import mean_squared_error
from xgboost import XGBRegressor

X = train.copy()
y = X.pop('Listening_Time_minutes')
X_test = test.copy()

oof_df = pd.DataFrame()
test_pred_df = pd.DataFrame()
result_df = pd.DataFrame()

n_folds = 7
kf = KFold(n_splits=n_folds, shuffle=True, random_state=34)

def add_models(model, name):
    oof_pred = cross_val_predict(model, X, y, cv=kf)
    rmse = np.sqrt(mean_squared_error(y, oof_pred))
    print(f'{name} RMSE {rmse:,.6f} ')

    oof_df[name] = oof_pred

    model.fit(X, y - y.mean())
    test_pred_df[name] = model.predict(X_test + y.mean())

    result_df.at[name, 'rmse'] = rmse
        

In [31]:
xgb = XGBRegressor(
        n_estimators=700, 
        learning_rate=0.07, 
        max_depth=11,
        colsample_bytree=0.5,
        subsample=0.8,
        device='cuda',
        tree_method='gpu_hist',
        eval_metric='rmse',
        random_state=34
    )

add_models(xgb, 'xgb')

xgb RMSE 12.718235 


In [32]:
from catboost import CatBoostRegressor

cat = CatBoostRegressor(
    iterations=1000,
    learning_rate= 0.06655423,  
    depth=9, 
    l2_leaf_reg=2.21012489857, 
    rsm=1.0,
    bagging_temperature=0.446987,
    random_strength=0.8499895,
    task_type='GPU',
    verbose=0,
)

add_models(cat, 'cat')

cat RMSE 12.926312 


In [33]:
from lightgbm import LGBMRegressor

lgm = LGBMRegressor(
    n_estimators=829, 
    learning_rate=0.085995, 
    max_depth=7,
    reg_lambda=1, 
    min_child_samples=4,
    num_leaves=31,
    verbose=-1,
    device='gpu',
)

add_models(lgm, 'lgm')

lgm RMSE 12.914261 


In [34]:
from sklearn.linear_model import BayesianRidge

meta_model = BayesianRidge()
oof_pred = cross_val_predict(meta_model, oof_df, y, cv=kf)
rmse = np.sqrt(mean_squared_error(y, oof_pred))
meta_model.fit(oof_df, y)

BayesianRidge()

## Final Submission

In [35]:
y_pred = pd.Series(meta_model.predict(test_pred_df), index=test_pred_df.index)
sub = pd.read_csv('/kaggle/input/playground-series-s5e4/sample_submission.csv')
sub['Listening_Time_minutes'] = y_pred
sub.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
